In [155]:
total = len(data1["feeds"][0]['LASS4U'])
timestamp = []
data123 = []
for i in range(total):
    tmp = list(data1["feeds"][0]['LASS4U'][i].keys())[0].split(':')[0]
    if tmp not in timestamp:
        timestamp.append(tmp)
        data123.append(data1["feeds"][0]['LASS4U'][i])


In [6]:
import csv
import json
import sys, pymongo
from urllib import request
import datetime

# 與 MongoDB連線
client = pymongo.MongoClient(host="localhost", port=27017)
# 指定為 test 資料庫
db = client.pm25
# 指定 temp_humidity 集合, MongoDB的每個資料庫又包含許多集合(collection), 類似於關聯性資料庫中的表
collection = db.pm25_2019

# 用戶代理瀏覽器種類
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 ' \
             'Safari/537.36 '
headers = {'User-Agent': user_agent}                                       # 請求網頁標頭，用字典型態存放

t = "2019-12-25"
#抓前一天的資料
#times = datetime.datetime.now() + datetime.timedelta(days = -1)
#t = times.strftime("%Y-%m-%d")
#t = datetime.datetime.now().strftime("%Y-%m-%d")
print(t)
url = 'https://pm25.lass-net.org/data/history-date.php?device_id=FT2_0041&date=%s'%(t)  # 網址
req = request.Request(url=url, headers=headers)                            # 請求內容
res = request.urlopen(req)  # 打開網頁（此網頁內容即為json）
jdata = res.read()          # 讀取網頁
data1 = json.loads(jdata)   # 用json的loads方法儲存為json格式（此為dict物件），，並轉為data1物件

total = len(data1["feeds"][0]['LASS4U'])
timestamp = []
data = []
for i in range(total):
    tmp = list(data1["feeds"][0]['LASS4U'][i].keys())[0].split(':')[0]
    if tmp not in timestamp:
        timestamp.append(tmp)
        data.append(data1["feeds"][0]['LASS4U'][i])

        
for j in range(len(data)):        
    data24 = list(data[j].values())[0]
    json = {"device_id": data24["device_id"],"date": data24["date"],"time": data24["time"],"timestamp": data24["timestamp"], "PM25": data24["s_d0"],"Temperature": data24["s_t2"], "Humidity": data24["s_h2"]}
    #print(json)
    # 將資料存入 mongodb
    # 存入單筆
    result = collection.insert_one(json)
    # 存入多筆
    #result = collection.insert_many()
    print(result)
    

2019-12-25


In [49]:
t = datetime.datetime.now() + datetime.timedelta(days = -1)
print(t.strftime("%Y-%m-%d"))

2019-12-13


In [8]:
import csv
import json
import sys, pymongo ,pymysql
from urllib import request
import datetime


#與MySQL建立連線
conn = pymysql.connect(host='localhost',port=3306,user='root',password='123456',db='pm25')
cursor = conn.cursor()


# 用戶代理瀏覽器種類
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 ' \
             'Safari/537.36 '
headers = {'User-Agent': user_agent}                                       # 請求網頁標頭，用字典型態存放

t = "2019-12-16"
#抓前一天的資料
#times = datetime.datetime.now() + datetime.timedelta(days = -1)
#t = times.strftime("%Y-%m-%d")
print(t)
url = 'https://pm25.lass-net.org/data/history-date.php?device_id=FT2_0041&date=%s'%(t)  # 網址
req = request.Request(url=url, headers=headers)                            # 請求內容
res = request.urlopen(req)  # 打開網頁（此網頁內容即為json）
jdata = res.read()          # 讀取網頁
data1 = json.loads(jdata)   # 用json的loads方法儲存為json格式（此為dict物件），，並轉為data1物件

total = len(data1["feeds"][0]['LASS4U'])
timestamp = []
data = []
for i in range(total):
    tmp = list(data1["feeds"][0]['LASS4U'][i].keys())[0].split(':')[0]
    if tmp not in timestamp:
        timestamp.append(tmp)
        data.append(data1["feeds"][0]['LASS4U'][i])

        
for j in range(len(data)):        
    data24 = list(data[j].values())[0]
    json = {"device_id": data24["device_id"],"date": data24["date"],"time": data24["time"],"timestamp": data24["timestamp"], "PM25": data24["s_d0"],"Temperature": data24["s_t2"], "Humidity": data24["s_h2"]}
    print(json)
    cursor.execute("INSERT INTO pm25_2019(timestamp,date,time,device_id,Temperature,Humidity,PM25) VALUES('{0}','{1}','{2}','{3}','{4}','{5}','{6}');".format(data24["timestamp"],data24["date"],data24["time"],data24["device_id"],data24["s_t2"],data24["s_h2"],data24["s_d0"]))
    conn.commit()
    
cursor.close() 
conn.close()

2019-12-16
{'device_id': 'FT2_0041', 'date': '2019-12-16', 'time': '00:00:14', 'timestamp': '2019-12-16T00:00:14Z', 'PM25': 29.0, 'Temperature': 21.48, 'Humidity': 68.41}
{'device_id': 'FT2_0041', 'date': '2019-12-16', 'time': '01:00:19', 'timestamp': '2019-12-16T01:00:19Z', 'PM25': 22.0, 'Temperature': 22.18, 'Humidity': 69.33}
{'device_id': 'FT2_0041', 'date': '2019-12-16', 'time': '02:00:04', 'timestamp': '2019-12-16T02:00:04Z', 'PM25': 28.0, 'Temperature': 22.53, 'Humidity': 69.05}
{'device_id': 'FT2_0041', 'date': '2019-12-16', 'time': '03:00:06', 'timestamp': '2019-12-16T03:00:06Z', 'PM25': 21.0, 'Temperature': 22.86, 'Humidity': 68.19}
{'device_id': 'FT2_0041', 'date': '2019-12-16', 'time': '04:00:08', 'timestamp': '2019-12-16T04:00:08Z', 'PM25': 16.0, 'Temperature': 22.77, 'Humidity': 68.95}
{'device_id': 'FT2_0041', 'date': '2019-12-16', 'time': '05:00:05', 'timestamp': '2019-12-16T05:00:05Z', 'PM25': 17.0, 'Temperature': 22.82, 'Humidity': 67.4}
{'device_id': 'FT2_0041', 'dat

In [111]:
times = datetime.datetime.now() + datetime.timedelta(days = -1)
t = times.strftime("%Y-%m-%d")

In [112]:
print(t)

2019-12-14


In [124]:
datetime.datetime.now()

datetime.datetime(2019, 12, 22, 9, 0, 0, 729289)

In [126]:
import csv
import json
import sys, pymongo ,pymysql
from urllib import request
import datetime


#與MySQL建立連線
conn = pymysql.connect(host='192.168.0.110',port=3306,user='dbuser',password='aabb1234',db='my_pm25_db')
cursor = conn.cursor()

InternalError: (1130, "Host '192.168.0.111' is not allowed to connect to this MySQL server")